In [61]:
from llama_cpp import Llama
import chromadb
import warnings
warnings.filterwarnings('ignore')
from transformers import pipeline

# onderstaande koste mij ongeveer een halfuur
# pip install llama_cpp-python


In [62]:
documents = []
metadata = []
ids = []
with open('data/spells.txt', 'r', encoding='utf-8') as spells:
    for line in spells:
        line=line.split(" - ") #spells are separated by their description by a " - "
        metadata.append({'spell': line[0]}) #use the spell name as metadata. ChromaDB requires a dictionary here
        documents.append(line[1]) #use the description as documents
        ids.append(line[0]) #we also need an ID so use the spell name again.
print(ids)

['Aberto', 'Accio', 'Aguamenti', 'Alohomora', 'Anapneo', 'Aparecium', 'Apparate', 'Ascendio', 'Avada Kedavra', 'Avis', 'Bat-Bogey Hex', 'Bombardo', 'Brackium Emendo', 'Capacious Extremis', 'Confundo', 'Conjunctivitis Curse', 'Crinus Muto', 'Crucio', 'Diffindo', 'Disillusionment Charm', 'Disapparate', 'Engorgio', 'Episkey', 'Expecto patronum', 'Erecto', 'Evanesco', 'Expelliarmus', 'Ferula', 'Fidelius Charm', 'Fiendfyre Curse', 'Finite Incantatem', 'Furnunculus Curse', 'Geminio', 'Glisseo', 'Homenum Revelio', 'Homonculus Charm', 'Immobulus', 'Impedimenta', 'Incarcerous', 'Imperio', 'Impervius', 'Incendio', 'Langlock', 'Legilimens', 'Levicorpus', 'Locomotor Mortis', 'Lumos', 'Morsmordre', 'Mucus Ad Nauseam', 'Muffliato', 'Nox', 'Obliviate', 'Obscuro', 'Oculus Reparo', 'Oppugno', 'Petrificus Totalus', 'Periculum', 'Piertotum Locomotor', 'Protean Charm', 'Protego', 'Reducto', 'Reducio', 'Renneverate', 'Reparifors', 'Reparo', 'Rictusempra', 'Riddikulus', 'Scourgify', 'Sectumsempra', 'Serpens

In [63]:
client = chromadb.Client()

# create or open collection named 'spells' 
spells_collection = client.get_or_create_collection("spells")

# add spells to the collection, ignored for ids that already exist 
spells_collection.add(
    documents=documents,
    metadatas=metadata,
    ids=ids
)

Add of existing embedding ID: Aberto
Add of existing embedding ID: Accio
Add of existing embedding ID: Aguamenti
Add of existing embedding ID: Alohomora
Add of existing embedding ID: Anapneo
Add of existing embedding ID: Aparecium
Add of existing embedding ID: Apparate
Add of existing embedding ID: Ascendio
Add of existing embedding ID: Avada Kedavra
Add of existing embedding ID: Avis
Add of existing embedding ID: Bat-Bogey Hex
Add of existing embedding ID: Bombardo
Add of existing embedding ID: Brackium Emendo
Add of existing embedding ID: Capacious Extremis
Add of existing embedding ID: Confundo
Add of existing embedding ID: Conjunctivitis Curse
Add of existing embedding ID: Crinus Muto
Add of existing embedding ID: Crucio
Add of existing embedding ID: Diffindo
Add of existing embedding ID: Disillusionment Charm
Add of existing embedding ID: Disapparate
Add of existing embedding ID: Engorgio
Add of existing embedding ID: Episkey
Add of existing embedding ID: Expecto patronum
Add of e

In [64]:
# En dit koste mij ook weer een halfuur

llm = Llama.from_pretrained(
	repo_id="BramVanroy/GEITje-7B-ultra-GGUF",
	filename="geitje-7b-ultra-q8_0.gguf",
)

llm.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": "What is the capital of France?"
		}
	]
)

llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from C:\Users\raymo\.cache\huggingface\hub\models--BramVanroy--GEITje-7B-ultra-GGUF\snapshots\00da1c06460774d6982d608e495dac102ead3ec1\.\geitje-7b-ultra-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = geitje-7b-ultra
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 32768
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:            

{'id': 'chatcmpl-adeae708-f7ec-4e85-9916-e467b0a35d95',
 'object': 'chat.completion',
 'created': 1728984462,
 'model': 'C:\\Users\\raymo\\.cache\\huggingface\\hub\\models--BramVanroy--GEITje-7B-ultra-GGUF\\snapshots\\00da1c06460774d6982d608e495dac102ead3ec1\\.\\geitje-7b-ultra-q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Het capital van Frankrijk is Parijs.'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 23, 'completion_tokens': 11, 'total_tokens': 34}}

In [65]:
user_input = input("Wat is jouw vraag?:")
results = spells_collection.query(
    query_texts=[user_input],
    n_results=1
)
results

{'ids': [['Impedimenta']],
 'distances': [[0.7246537804603577]],
 'metadatas': [[{'spell': 'Impedimenta'}]],
 'embeddings': None,
 'documents': [['Een tijdelijke vloek die de beweging van het doelwit vertraagt\n']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [66]:
result=results['metadatas'][0][0]['spell']+" - " + results['documents'][0][0][:-1]
result

'Impedimenta - Een tijdelijke vloek die de beweging van het doelwit vertraagt'

In [67]:
prompt = f"""
voorgestelde spreuk: {result}
User input: {user_input}
stel een spreuk voor op basis van de voorgestelde spreuk en de user input.
"""
print("Prompt: ", prompt)
inference = llm(prompt, max_tokens=100)
generated_text = inference['choices'][0]['text']
print("Inference: ", generated_text)

Prompt:  
voorgestelde spreuk: Impedimenta - Een tijdelijke vloek die de beweging van het doelwit vertraagt
User input: hoe kan ik iets laten vliegen
stel een spreuk voor op basis van de voorgestelde spreuk en de user input.



llama_perf_context_print:        load time =   11267.35 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    81 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    79 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   38752.53 ms /   160 tokens


Inference:  Impedimenta - Een tijdelijke vloek die de beweging van het doelwit vertraagt, opdat het even blijft zweven, zodat het gemakkelijker is voor degene die het oproept om nauwkeurig te mikken en de vloek te richten.
